In [1]:
import pandas as pd
import math
from collections import Counter, defaultdict
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /Users/chris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/chris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Part I: Indexing

In [3]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

def build_terms(line):    
    line = re.sub(r"[^a-zA-Z\s-]", " ", line) # removing punctuation and digits
    line = line.lower() # transform in lowercase
    line = line.split() # tokenization
    line = [w for w in line if w and w not in stop_words and w not in {"nan","none","null"}]
    line = [stemmer.stem(w) for w in line] # stemming

    return line

### 1. Build inverted index

In [4]:
# toy dataset
data = {
    "pid": [1, 2, 3],
    "doc_text": [
        "nike running shoes",
        "adidas running jacket",
        "nike jacket blue"
    ]
}

df = pd.DataFrame(data)
print(df)

   pid               doc_text
0    1     nike running shoes
1    2  adidas running jacket
2    3       nike jacket blue


In [5]:
# invert index
def build_inverted_index(df):
    index = {}

    for _, row in df.iterrows():
        doc_id = row["pid"]
        terms = row["doc_text"].split() # tokenize
        for term in set(terms): # duplicates
            if term not in index:
                index[term] = [doc_id]
            else:
                index[term].append(doc_id)

    return index

In [6]:
index = build_inverted_index(df)

for term, docs in index.items():
    print(f"{term}: {docs}")

nike: [1, 3]
running: [1, 2]
shoes: [1]
jacket: [2, 3]
adidas: [2]
blue: [3]


In [7]:
# search
def search_and(query, index):
    terms = build_terms(query)
    result = None

    for term in terms:
        if term in index:
            docs = set(index[term])
            result = docs if result is None else result & docs
        else:
            return []

    return sorted(result)

### 2. Propose test queries

In [8]:
queries = [
    "nike running shoes",
    "adidas jacket",
    "women cotton dress",
    "black leather bag",
    "discount sport shoes"
]

### 3. Rank your results

$$tf_{t,d} = \dfrac{N_{t,d}}{||D||}\tag{1}, \qquad idf_t = log\dfrac{N}{df_t} \qquad w_{t,d}=tf_{t,d}\cdot idf_t$$

In [9]:
# weights
def build_idf(df): # idf
    N = len(df) # total number of docs in a collection
    idf = {} # dict to save idf(t)

    for text in df["doc_text"]:
        words = set(text.split())
        for w in words:
            idf[w] = idf.get(w, 0) + 1
    
    for w in idf: # idf(t) = log(N / df(t))
        idf[w] = math.log(N / idf[w])

    return idf

def build_tfidf(df, idf): # tf-idf
    docs = {}

    for _, row in df.iterrows():
        pid = row["pid"]
        terms = build_terms(row["doc_text"])
        tf = Counter(terms)
        L = len(terms) or 1
        vec = {t: (tf[t]/L) * idf.get(t, 0.0) for t in tf}
        norm = math.sqrt(sum(v*v for v in vec.values())) or 1.0
        docs[pid] = {t: v/norm for t, v in vec.items()}
        
    return docs

In [10]:
# ranking
def query_vector(query, idf):
    terms = build_terms(query)
    tf = Counter(terms)
    L = len(terms) or 1
    vec = {t: (tf[t]/L) * idf.get(t, 0.0) for t in tf}
    norm = math.sqrt(sum(v*v for v in vec.values())) or 1.0

    return {t: v/norm for t, v in vec.items()}

def score_cosine(doc_vec, q_vec): # cosine similarity
    s = 0.0

    for t, wq in q_vec.items():
        wd = doc_vec.get(t)
        if wd:
            s += wd * wq

    return s

def rank_tfidf(docs, idf, query):
    qv = query_vector(query, idf)
    scores = [(pid, score_cosine(vec, qv)) for pid, vec in docs.items()]
    scores.sort(key=lambda x: x[1], reverse=True)
    
    return [(pid, round(s, 4)) for pid, s in scores if s > 0]

In [11]:
def print_rankings(df, queries):
    if isinstance(queries, str):
        queries = [queries]
    idf  = build_idf(df)
    docs = build_tfidf(df, idf)

    for qi, q in enumerate(queries, 1):
        res = rank_tfidf(docs, idf, q)
        print(f"\n[q{qi}]: {q}")
        if not res:
            print("no results")
            continue
        for i, (pid, score) in enumerate(res, 1):
            text = df.loc[df["pid"] == pid, "doc_text"].values[0]
            print(f"{i}. {pid} | {score:.4f} | {text}")

In [12]:
print_rankings(df, queries)


[q1]: nike running shoes
1. 1 | 1.0000 | nike running shoes
2. 3 | 0.3272 | nike jacket blue

[q2]: adidas jacket
1. 2 | 1.0000 | adidas running jacket
2. 3 | 0.3272 | nike jacket blue

[q3]: women cotton dress
no results

[q4]: black leather bag
no results

[q5]: discount sport shoes
no results


# Part II: Evaluation

### 1. Evaluation metrics

- Precision@K (P@K): Of the K items returned, what fraction are relevant?
- Recall@K (R@K): Of all the relevant items that exist, what fraction did I retrieve in the top K?
- Average Precision@K (AP@K): Average of the precisions at the positions where a relevant item appears (up to K).
- F1@K: Harmonic mean between P@K and R@K.
- MAP: Average (over queries) of AP (or AP@K).
- MRR: Average (over queries) of 1/(position of the first relevant item).
- NDCG@K: Higher scores are given to those who place relevant items higher in the ranking. DCG normalized by the maximum possible (IDCG).

In [ ]:
# p@k
def precision_at_k():

In [ ]:
# r@k
def recall_at_k():

In [ ]:
# AP
def average_precision_at_k():

In [ ]:
# F1@k
def f1_at_k():

In [ ]:
# MAP@k
def map_at_k():

In [ ]:
# MRR
def MRR_at_k():

In [ ]:
# nDCG@K
def ndcg_at_k():